In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.2 MB/s eta 0:00:00


## Импорты

In [2]:
from ultralytics import YOLO
import torch
import os
import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, f1_score, auc
from PIL import Image

from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import json
from collections import defaultdict

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Использую данные, которые загрузила на гугл диск

In [ ]:
# Подключаюсь к гугл диску
%cd /content/drive/MyDrive/

/content/drive/MyDrive


## Обучение YOLO под здадачу детекции логотипов T-банка

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("CUDA not available. Training on CPU")

Using device: cuda
GPU: Tesla T4
CUDA memory: 14.7 GB


In [ ]:
model = YOLO('yolov8n.pt')

training_config = {
    'data': '/content/data.yaml',
    'epochs': 50,
    'imgsz': 640,
    'batch': 16 if device == 'cuda' else 4,
    'device': device,
    'name': 'tbank_logo_detector',
    'patience': 10,
    'optimizer': 'auto',
    'lr0': 0.01,
    'lrf': 0.01,
    'weight_decay': 0.0005,
    'save': True,
    'save_period': 10,
    'val': True,
}

print("Starting training...")
print(f"Config: {training_config}")

results = model.train(**training_config)

print("Training completed!")

Starting training...
Config: {'data': '/content/data.yaml', 'epochs': 50, 'imgsz': 640, 'batch': 16, 'device': 'cuda', 'name': 'tbank_logo_detector', 'patience': 10, 'optimizer': 'auto', 'lr0': 0.01, 'lrf': 0.01, 'weight_decay': 0.0005, 'save': True, 'save_period': 10, 'val': True}
Ultralytics 8.3.199 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01

## Нарисовала коробки для тестовых изображений

In [ ]:
model = YOLO('/content/drive/MyDrive/t_bank_logo_detection/runs/detect/tbank_logo_detector1/weights/best.pt')
test_images = list(Path('/content/drive/MyDrive/t_bank_logo_detection/prepared/images/test').glob('*.jpg'))

for img_path in test_images:
    results = model(str(img_path), conf=0.25)

    plotted_image = results[0].plot()

    output_path = f"/content/drive/MyDrive/t_bank_logo_detection/runs/detect/result1/simple_{img_path.name}"
    cv2.imwrite(output_path, plotted_image)
    print(f"Saved: {output_path}")


image 1/1 /content/drive/MyDrive/prepared/images/test/0b87a1f27d6009df2877202c56d074c1.jpg: 448x640 (no detections), 38.6ms
Speed: 2.1ms preprocess, 38.6ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)
Saved: /content/drive/MyDrive/runs/detect/result1/simple_0b87a1f27d6009df2877202c56d074c1.jpg

image 1/1 /content/drive/MyDrive/prepared/images/test/0a6d95b8696d5f95593a661733fdbf64.jpg: 352x640 (no detections), 153.2ms
Speed: 2.7ms preprocess, 153.2ms inference, 1.7ms postprocess per image at shape (1, 3, 352, 640)
Saved: /content/drive/MyDrive/runs/detect/result1/simple_0a6d95b8696d5f95593a661733fdbf64.jpg

image 1/1 /content/drive/MyDrive/prepared/images/test/0ad6e17022db5347dd7c9cf4c3b541ad.jpg: 512x640 (no detections), 52.9ms
Speed: 3.9ms preprocess, 52.9ms inference, 0.7ms postprocess per image at shape (1, 3, 512, 640)
Saved: /content/drive/MyDrive/runs/detect/result1/simple_0ad6e17022db5347dd7c9cf4c3b541ad.jpg

image 1/1 /content/drive/MyDrive/prepared/images/

## Предсказание метрик

In [8]:
model = YOLO('/content/drive/MyDrive/t_bank_logo_detection/runs/detect/tbank_logo_detector1/weights/best.pt')

metrics = model.val(
    data='/content/drive/MyDrive/t_bank_logo_detection/data.yaml',
    split='test',
    conf=0.25,
    iou=0.6,
    device='cpu'
)

print("\n" + "="*50)
print("МЕТРИКИ")
print("="*50)
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

metrics_dict = {
    'map50': float(metrics.box.map50),
    'map50_95': float(metrics.box.map),
    'precision': float(metrics.box.mp),
    'recall': float(metrics.box.mr),
    'f1_score': float(2 * metrics.box.mp * metrics.box.mr / (metrics.box.mp + metrics.box.mr))
                if (metrics.box.mp + metrics.box.mr) > 0 else 0
}

with open('/content/drive/MyDrive/t_bank_logo_detection/runs/detect/result1/yolo_metrics.json', 'w') as f:
    json.dump(metrics_dict, f, indent=4)

print(metrics_dict)

Ultralytics 8.3.201 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 1.5±1.9 ms, read: 9.3±14.5 MB/s, size: 29.5 KB)
val: Scanning /content/drive/MyDrive/t_bank_logo_detection/prepared/labels/test... 164 images, 146 backgrounds, 2 corrupt: 100% ━━━━━━━━━━━━ 164/164 114.6it/s 1.4s
val: /content/drive/MyDrive/t_bank_logo_detection/prepared/images/test/0acb0564e5fe7da686d3d74a6b322974.jpg: ignoring corrupt image/label: negative class labels or coordinate [  -0.058472]
val: /content/drive/MyDrive/t_bank_logo_detection/prepared/images/test/0e7baf21b5337eccf781f15f2126fe9e.jpg: ignoring corrupt image/label: negative class labels or coordinate [  -0.049643]
val: New cache created: /content/drive/MyDrive/t_bank_logo_detection/prepared/labels/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 11/11 0.

## Надо проверить, что функция predict работает

In [ ]:
def predict_image(image: Image.Image, confidence_threshold: float = 0.5, model=None):
    """
    Предсказание через YOLO
    """
    opencv_image = np.array(image)
    opencv_image = opencv_image[:, :, ::-1].copy()

    results = model(opencv_image, conf=confidence_threshold)

    detections = []
    for result in results:
        if result.boxes is not None:
            for box in result.boxes:
                x_min, y_min, x_max, y_max = (
                    box.xyxy[0].cpu().numpy().astype(int).tolist()
                )
                confidence = box.conf[0].cpu().numpy().item()

                detections.append(
                    {
                        "x_min": x_min,
                        "y_min": y_min,
                        "x_max": x_max,
                        "y_max": y_max,
                        "confidence": confidence,
                    }
                )

    return detections

image_path = "/content/drive/MyDrive/t_bank_logo_detection/prepared/images/test/0e44ed402d915f7f5a9b38dd98494142.jpg"
image = Image.open(image_path)
model = YOLO('/content/drive/MyDrive/t_bank_logo_detection/runs/detect/tbank_logo_detector1/weights/best.pt')

result = predict_image(image, confidence_threshold = 0.25, model=model)
print(result)


0: 640x544 1 t-bank-logo, 7.7ms
Speed: 2.4ms preprocess, 7.7ms inference, 216.7ms postprocess per image at shape (1, 3, 640, 544)
[{'x_min': 134, 'y_min': 587, 'x_max': 314, 'y_max': 720, 'confidence': 0.4697149395942688}]
